In [17]:
import pandas as pd
import numpy as np 
import sklearn as sk 
import matplotlib.pyplot as plt


In [18]:
df=pd.read_csv('train.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115475 entries, 0 to 115474
Data columns (total 13 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   case_id                         115475 non-null  object
 1   equipment_id                    115475 non-null  object
 2   completion_date                 115475 non-null  object
 3   action_recommendation_id        115475 non-null  object
 4   action_recommendation_type      115475 non-null  object
 5   action_recommendation_category  115475 non-null  object
 6   equipment_area                  115475 non-null  object
 7   usage_type                      115475 non-null  object
 8   speed_category                  115475 non-null  int64 
 9   load_category                   115475 non-null  int64 
 10  floors_category                 115475 non-null  int64 
 11  equipment_category              115475 non-null  object
 12  feedback                      

In [20]:
df.isnull().sum()

case_id                           0
equipment_id                      0
completion_date                   0
action_recommendation_id          0
action_recommendation_type        0
action_recommendation_category    0
equipment_area                    0
usage_type                        0
speed_category                    0
load_category                     0
floors_category                   0
equipment_category                0
feedback                          0
dtype: int64

In [21]:
df.head(5)

,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,b7c775ad-4ebe-4848-9c53-37e7c5658e21,ele0000754,2018-10-03,ar00000174,art01,arc03,ga00000803,ut012,7,6,8,tp006,1
1,b177eefd-3946-4949-9699-0a91879350f9,ele0000789,2018-10-03,ar00000248,art02,arc03,ga00000269,ut005,4,6,2,tp001,1
2,fc2d568a-c53c-43ef-8871-a49ec22ab3b1,ele0001227,2018-10-03,ar00000158,art02,arc03,ga00000064,ut011,6,6,7,tp006,1
3,8e645922-1268-4c6b-ae6b-7b1605689cca,ele0001754,2018-10-03,ar00000105,art01,arc03,ga00000662,ut005,8,7,8,tp014,1
4,b44f10b1-9238-44db-8f0c-2d68e8c015a7,ele0002087,2018-10-03,ar00000148,art02,arc03,ga00001256,ut011,7,6,8,tp013,1


In [23]:
#convertire les variable qualitative
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["case_id"] = ord_enc.fit_transform(df[["case_id"]])
df["equipment_id"] = ord_enc.fit_transform(df[["equipment_id"]])
df["completion_date"] = ord_enc.fit_transform(df[["completion_date"]])
df["action_recommendation_id"] = ord_enc.fit_transform(df[["action_recommendation_id"]])
df["action_recommendation_type"] = ord_enc.fit_transform(df[["action_recommendation_type"]])
df["action_recommendation_category"] = ord_enc.fit_transform(df[["action_recommendation_category"]])
df["equipment_area"] = ord_enc.fit_transform(df[["equipment_area"]])
df["usage_type"] = ord_enc.fit_transform(df[["equipment_area"]])
df["equipment_category"] = ord_enc.fit_transform(df[["equipment_category"]])

df.head(5)

,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,53095.0,753.0,0.0,173.0,0.0,2.0,802.0,802.0,7,6,8,6.0,1
1,51324.0,788.0,0.0,247.0,1.0,2.0,268.0,268.0,4,6,2,1.0,1
2,72877.0,1226.0,0.0,157.0,1.0,2.0,63.0,63.0,6,6,7,6.0,1
3,41103.0,1753.0,0.0,104.0,0.0,2.0,661.0,661.0,8,7,8,14.0,1
4,52110.0,2086.0,0.0,147.0,1.0,2.0,1255.0,1255.0,7,6,8,13.0,1


In [35]:
y= df['feedback'] #récupérer la colonne de target dans Y
X= df.drop('feedback', axis=1) #Récupérez le reste des colonnes comme features

In [36]:
from sklearn.preprocessing import scale #en charge la fonction de normalisation à partir de la bibliothèque sklearn
XX = scale(X) # normaliser les données

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, stratify=y)

In [38]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train,y_train)

y_pred=knn.predict(X_test)
#Calculer les erreurs MAE, MSE, RMSE
print('fP :',mean_absolute_error(y_test,y_pred))
print('fn: ', mean_squared_error(y_test,y_pred))
print('RMSE', np.sqrt(mean_squared_error(y_test,y_pred)))

fP : 0.19389478242043737
fn:  0.12835871160183784
RMSE 0.35827184036962473


In [39]:
#Charger le grid search qui cherche les paramètres optimaux d'un modèle
from sklearn.model_selection import GridSearchCV
#Charger le classificateur KNN
from sklearn.neighbors import KNeighborsClassifier
grid_params={
    'n_neighbors':[3,5,7,9,11],
    'metric':['euclidean','manhattan']
}

#Créer un objet grid search avec les paramètres et le modèle à tester
gs=GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    cv=5 #nombre de folds dans la cross validation
)
gs_results=gs.fit(X_train, y_train) #lancer le grid search et la cross validation

In [40]:
gs_results.best_params_

{'metric': 'manhattan', 'n_neighbors': 11}

In [41]:
#Créer une instance knn avec les paramètres optimaux trouvés
knn= KNeighborsClassifier(n_neighbors=3,metric='manhattan')
#Lancer l'apprentissage
knn.fit(X_train, y_train)#entraîner le modèles
y_pred=knn.predict(X_test) #evaluer le modèles

In [42]:
#afficher la précision et le rappel
from sklearn.metrics import precision_score, recall_score
print('precision=',precision_score(y_test,y_pred))
print('recall=',recall_score(y_test,y_pred))

precision= 0.8885554683532616
recall= 0.9495313270843612


In [43]:
#Afficher la matrice de confusion
from sklearn.metrics import  confusion_matrix
confusion_matrix(y_test, y_pred)


array([[  822,  4828],
       [ 2046, 38494]], dtype=int64)

In [2]:
#oui ce changement a un impact sur les performances du classificateur , désormais il detecte plus faut positf et de faux negatifs meme si ils restent negligable et la precision et le recall ont eux aussi légerement diminué et ceci a cause de l ajout de cas d'anomalie dans la partie training ce qui a rendu notre modéle plus correct .